# 1 i) Initial Setup of Classes

In [2]:
from collections import deque

class Analyst:
    def __init__(self, name, rating, workload=0, max_workload=40, available=True):
        self.name = name
        self.rating = rating  #scored 1-5
        self.workload = workload #hours assigned already default 0 
        self.max_workload = max_workload #maximum hours they can work default 40
        self.available = available # True if available for new tasks
        self.tasks = [] # List of tasks assigned to the analyst

class Task:
    def __init__(self, name, importance, effort):
        self.name = name
        self.importance = importance  # scored 1-5
        self.effort = effort # hours required to complete the task
        self.assigned_to = None

## 1 ii) dummy data

In [10]:
analysts = [
    Analyst("Alice", 5, 35, 40),
    Analyst("Bob", 4, 30, 40),
    Analyst("Charlie", 3, 20, 40),
    Analyst("Dana", 2, 10, 40),
    Analyst("Eli", 1, 5, 40),
    Analyst("Josh", 5, 0, 40)
]

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
]

## 2 i) starter function

In [8]:
def assign_tasks(analysts, tasks):
    task_queue = deque(sorted(tasks, key=lambda t: (-t.importance, t.effort)))
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                best_analyst = analyst
                break
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        else:
            print(f"No available analyst for {task.name}")
            
assign_tasks(analysts, tasks)

Assigned Task2 to Josh
Assigned Task4 to Josh
Assigned Task3 to Charlie
Assigned Task1 to Bob
Assigned Task5 to Dana


## 2 ii) reassign tasks in event of illness

In [ ]:
analysts = [
    Analyst("Alice", 5, 35, 40),
    Analyst("Bob", 4, 30, 40),
    Analyst("Charlie", 3, 20, 40),
    Analyst("Dana", 2, 10, 40),
    Analyst("Eli", 1, 5, 40),
    Analyst("Josh", 5, 0, 40)
]

tasks = [
    Task("Task1", 2, 10),
    Task("Task2", 5, 15),
    Task("Task3", 3, 20),
    Task("Task4", 4, 25),
    Task("Task5", 1, 30),
]

analysts.pop(2) #Charlie is ill
assign_tasks(analysts,tasks)

Assigned Task2 to Josh
Assigned Task4 to Josh
Assigned Task3 to Dana
Assigned Task1 to Bob
Assigned Task5 to Eli


## 3 i) New rule: Analysts may not attempt tasks with higher importance level than their analyst level

#### Analysts are now dicts not lists to allow task reassignment in 3 ii)

In [82]:
def assign_tasks_max_level(analysts_dict:dict, tasks:list):
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}")

## 3 ii) task realocattion - if analyst unavailable, keeps all analysts with existing tasks where possible

In [87]:
def reassign_tasks_unavailability(analysts_dict:dict ,unavailable_analyst:str):
    analysts_dict[unavailable_analyst].available = False
    tasks_for_reassignment=analysts_dict[unavailable_analyst].tasks
    print('tasks for reassignment:',[task.name for task in tasks_for_reassignment])
    analysts = list(analysts_dict.values())
    task_queue = deque(sorted(tasks_for_reassignment, key=lambda t: (-t.importance, t.effort)))
    unnasigned_tasks=[]
    
    while task_queue:
        task = task_queue.popleft()
        best_analyst = None
        
        for analyst in sorted(analysts, key=lambda a: (-a.rating, a.workload)):
            if analyst.available and (analyst.workload + task.effort <= analyst.max_workload):
                if analyst.rating >= task.importance:
                    best_analyst = analyst
                else:
                    print(f"No available analyst level {task.importance} or above for {task.name}")
                    unnasigned_tasks.append(task)
                break
        #condition to find max hours left for analyst
        hours_remaining = deque(sorted(analysts, key=lambda a: (- a.max_workload + a.workload)))
        
        if best_analyst:
            best_analyst.tasks.append(task)
            best_analyst.workload += task.effort
            best_analyst.available = best_analyst.workload < best_analyst.max_workload
            task.assigned_to = best_analyst.name
            print(f"Assigned {task.name} to {best_analyst.name}")
        elif task.effort > (hours_remaining[0].max_workload-hours_remaining[0].workload):
            print(f"""No available analyst for {task.name}, The task requires {task.effort} hours, however the employee with 
                  the most available hours is {hours_remaining[0].name} who has {hours_remaining[0].max_workload-hours_remaining[0].workload} hours available, 
                  consider a temporary contractor""")
            unnasigned_tasks.append(task)
    if len(unnasigned_tasks) > 0:
        print(f"WARNING: Tasks unable to be assigned are {[task.name for task in unnasigned_tasks]}")
        
analysts_dict = {
    "Alice": Analyst("Alice", 5, 0, 40),
    "Bob": Analyst("Bob", 4, 0, 40),
    "Charlie": Analyst("Charlie", 3, 0, 40),
    "Dana": Analyst("Dana", 2, 0, 40),
    "Eli": Analyst("Eli", 1, 0, 40),
    "Josh":Analyst("Josh", 5, 0, 40)
}
assign_tasks_max_level(analysts_dict,tasks)
reassign_tasks_unavailability(analysts_dict,"Josh")

Assigned Task2 to Alice
Assigned Task5 to Josh
Assigned Task4 to Alice
Assigned Task3 to Bob
Assigned Task1 to Josh
Assigned Task6 to Bob
Assigned Task7 to Charlie
tasks for reassignment: ['Task5', 'Task1']
No available analyst level 5 or above for Task5
Assigned Task1 to Dana


### To do list:

##### Recursive feature which only reassignes tasks when neccecary
##### Optimise for minimal time wastage/reassignment